### Google Trends

In [23]:
import time
from datetime import datetime, timedelta
import pandas as pd
from pytrends.request import TrendReq
import os

# Function to generate daily intervals
def generate_daily_intervals(start_date, end_date):
    intervals = []
    while start_date < end_date:
        interval_end = min(start_date + timedelta(days=30), end_date)
        intervals.append((start_date.strftime("%Y-%m-%d"), interval_end.strftime("%Y-%m-%d")))
        start_date = interval_end + timedelta(days=1)
    return intervals

# Function to rescale data between overlapping intervals
def rescale_data(existing_data, new_data):
    overlap = existing_data.index.intersection(new_data.index)
    if not overlap.empty:
        # Calculate scaling factor using overlapping days
        scale_factor = existing_data.loc[overlap].mean() / new_data.loc[overlap].mean()
        # Rescale new data
        new_data *= scale_factor
        print(f"Rescaled new data by factor: {scale_factor}")
    return new_data

# Function to fetch data with backoff logic
def fetch_with_backoff(pytrends, keyword, interval, geo, retries=10000):
    delay = 0  # Initial delay in seconds
    for attempt in range(retries):
        try:
            print(f"Attempt {attempt + 1}/{retries}: Fetching data from {interval[0]} to {interval[1]}...")
            pytrends.build_payload([keyword], timeframe=f"{interval[0]} {interval[1]}", geo=geo)
            data = pytrends.interest_over_time()
            
            # Ensure data is not empty
            if not data.empty:
                return data
            else:
                print(f"No data available for {interval[0]} to {interval[1]}.")
                return pd.DataFrame()  # Return empty DataFrame

        except Exception as e:
            print(f"Error fetching data for {interval}: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # Exponential backoff
            else:
                print("Max retries reached. Skipping this interval.")
                return pd.DataFrame()  # Return empty DataFrame if max retries reached

# Fetch Google Trends data with improvements
def fetch_google_trends_data(keyword, start_date, end_date, geo="US", output_file="google_trends_data.csv"):
    pytrends = TrendReq(hl='en-US', tz=360, timeout=(10, 25))
    
    # Generate daily intervals (30 days max)
    intervals = generate_daily_intervals(start_date, end_date)

    if os.path.exists(output_file):
        existing_data = pd.read_csv(output_file, index_col=0, parse_dates=True)
    else:
        existing_data = pd.DataFrame()

    for interval in intervals:
        print(f"Processing interval: {interval[0]} to {interval[1]}...")
        data = fetch_with_backoff(pytrends, keyword, interval, geo)
        
        if not data.empty:
            data = data.drop(columns=['isPartial'], errors='ignore')
            data.index = pd.to_datetime(data.index)

            # Combine and rescale data
            data = rescale_data(existing_data, data)
            existing_data = pd.concat([existing_data, data])
            existing_data.sort_index(inplace=True)
            existing_data.to_csv(output_file)
            print(f"Data from {interval[0]} to {interval[1]} saved successfully.")
        else:
            print(f"Skipping interval {interval[0]} to {interval[1]} due to fetch issues.")

# Main script
if __name__ == "__main__":
    # Define parameters
    keyword = "bitcoin"
    start_date = datetime(2023, 3, 12)
    end_date = datetime(2024, 11, 16)
    geo = ''  # Leave empty for worldwide data
    output_file = "../data/raw/bitcoin_daily_google_trends.csv"

    # Fetch data
    fetch_google_trends_data(keyword, start_date, end_date, geo, output_file)
    print(f"Data fetching completed. All data saved to {output_file}.")

Processing interval: 2023-03-12 to 2023-04-11...
Attempt 1/10000: Fetching data from 2023-03-12 to 2023-04-11...
Data from 2023-03-12 to 2023-04-11 saved successfully.
Processing interval: 2023-04-12 to 2023-05-12...
Attempt 1/10000: Fetching data from 2023-04-12 to 2023-05-12...
Data from 2023-04-12 to 2023-05-12 saved successfully.
Processing interval: 2023-05-13 to 2023-06-12...
Attempt 1/10000: Fetching data from 2023-05-13 to 2023-06-12...
Data from 2023-05-13 to 2023-06-12 saved successfully.
Processing interval: 2023-06-13 to 2023-07-13...
Attempt 1/10000: Fetching data from 2023-06-13 to 2023-07-13...
Data from 2023-06-13 to 2023-07-13 saved successfully.
Processing interval: 2023-07-14 to 2023-08-13...
Attempt 1/10000: Fetching data from 2023-07-14 to 2023-08-13...
Data from 2023-07-14 to 2023-08-13 saved successfully.
Processing interval: 2023-08-14 to 2023-09-13...
Attempt 1/10000: Fetching data from 2023-08-14 to 2023-09-13...
Data from 2023-08-14 to 2023-09-13 saved succes

In [26]:
import pandas as pd

# Function to remove duplicate dates
def remove_duplicates(csv_file, output_file):
    # Load the data
    data = pd.read_csv(csv_file, index_col=0, parse_dates=True)
    
    # Sort the data by index (date)
    data.sort_index(inplace=True)
    
    # Drop duplicate indices, keeping only the first occurrence
    data = data[~data.index.duplicated(keep='first')]
    
    # Save the cleaned data back to the CSV file
    data.to_csv(output_file)
    print(f"Duplicates removed. Cleaned data saved to {output_file}.")

# Specify file paths
input_file = "../data/raw/bitcoin_daily_google_trends.csv"  # Input file with duplicates
output_file = "../data/processed/bitcoin_daily_google_trends.csv"  # Output file without duplicates

# Remove duplicates
remove_duplicates(input_file, output_file)

Duplicates removed. Cleaned data saved to bitcoin_daily_google_trends.csv.


In [27]:
import pandas as pd

# Function to rescale data between overlapping intervals
def rescale_data(data):
    rescaled_data = pd.DataFrame()
    intervals = data.groupby(data.index.year)  # Group data by year for easier processing
    previous_interval = None

    for year, interval in intervals:
        if previous_interval is not None:
            # Identify overlap
            overlap = previous_interval.index.intersection(interval.index)
            if not overlap.empty:
                # Calculate scaling factor
                scale_factor = previous_interval.loc[overlap].mean() / interval.loc[overlap].mean()
                interval *= scale_factor
                print(f"Year {year}: Rescaled interval by factor {scale_factor:.4f}")

        # Add the interval to the rescaled data
        rescaled_data = pd.concat([rescaled_data, interval])
        previous_interval = interval

    rescaled_data.sort_index(inplace=True)
    return rescaled_data

# Load collected data
input_file = "../data/processed/bitcoin_daily_google_trends.csv"
output_file = "../data/processed/bitcoin_daily_google_trends.csv"
data = pd.read_csv(input_file, index_col=0, parse_dates=True)

# Rescale data
rescaled_data = rescale_data(data)

# Save rescaled data
rescaled_data.to_csv(output_file)
print(f"Rescaled data saved to {output_file}.")

Rescaled data saved to bitcoin_rescaled_google_trends.csv.


In [31]:
import pandas as pd

# Function to add UTC time to date index
def format_date_with_utc(input_file, output_file):
    # Load the data
    data = pd.read_csv(input_file, index_col=0, parse_dates=True)
    
    # Ensure the index is a datetime object with UTC timezone
    data.index = data.index.tz_localize('UTC')
    
    # Save the updated data to a new CSV file
    data.to_csv(output_file, date_format='%Y-%m-%d %H:%M:%S%z')
    print(f"Date format updated with UTC. Saved to {output_file}.")

# Specify file paths
input_file = "../data/processed/bitcoin_daily_google_trends.csv"  # Input file without UTC time
output_file = "../data/processed/bitcoin_daily_google_trends.csv"  # Output file with UTC time

# Add UTC time to dates
format_date_with_utc(input_file, output_file)

Date format updated with UTC. Saved to bitcoin_utc_google_trends.csv.


### Oil and Gold Price

In [34]:
import yfinance as yf
import pandas as pd

# Define symbols for oil and gold
# For example, CL=F is the crude oil futures and GC=F is gold futures on Yahoo Finance
oil_symbol = "CL=F"
gold_symbol = "GC=F"

# Fetch data for the past 10 years
oil_data = yf.download(oil_symbol, start="2014-11-14", end="2024-11-17", interval="1d")
gold_data = yf.download(gold_symbol, start="2014-11-14", end="2024-11-17", interval="1d")

# Save to CSV or analyze directly
oil_data.to_csv("../data/raw/oil_prices.csv")
gold_data.to_csv("../data/raw/gold_prices.csv")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['CL=F']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))
[*********************100%***********************]  1 of 1 completed


In [6]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Load the cleaned data
gold_data = pd.read_csv("../data/raw/gold_prices.csv", parse_dates=["Date"], index_col="Date")
oil_data = pd.read_csv("../data/raw/oil_prices.csv", parse_dates=["Date"], index_col="Date")

# Retain only the Close prices
gold_data_cleaned = gold_data[['Close']].rename(columns={'Close': 'Gold_Close'})
oil_data_cleaned = oil_data[['Close']].rename(columns={'Close': 'Oil_Close'})

# Create a full date range for the data
full_date_range = pd.date_range(start=gold_data_cleaned.index.min(), end=gold_data_cleaned.index.max())

# Reindex to the full date range
gold_data_cleaned = gold_data_cleaned.reindex(full_date_range)
oil_data_cleaned = oil_data_cleaned.reindex(full_date_range)

# Ensure the index is named 'Date'
gold_data_cleaned.index.name = 'Date'
oil_data_cleaned.index.name = 'Date'

# Step 1: Forward Fill for Edge Cases
gold_data_cleaned = gold_data_cleaned.fillna(method='ffill')
oil_data_cleaned = oil_data_cleaned.fillna(method='ffill')

# Step 2: Linear Interpolation for Small Gaps
gold_data_cleaned = gold_data_cleaned.interpolate(method='linear')
oil_data_cleaned = oil_data_cleaned.interpolate(method='linear')

# Step 3: ARIMA-Based Imputation for Large Gaps
def arima_impute(series):
    if series.isnull().sum() > 0:  # Apply ARIMA only if there are still missing values
        model = ARIMA(series, order=(1, 1, 1))  # Adjust (p, d, q) based on data
        fitted_model = model.fit()
        series = series.fillna(fitted_model.fittedvalues)
    return series

gold_data_cleaned['Gold_Close'] = arima_impute(gold_data_cleaned['Gold_Close'])
oil_data_cleaned['Oil_Close'] = arima_impute(oil_data_cleaned['Oil_Close'])

# Combine the gold and oil data
combined_data = gold_data_cleaned.join(oil_data_cleaned, how='inner')

# Save the combined DataFrame to a CSV file
combined_data.to_csv("../data/processed/gold_oil_prices.csv")

# Display the result
print(combined_data.head())

                            Gold_Close  Oil_Close
Date                                             
2014-11-14 00:00:00+00:00  1185.000000  75.820000
2014-11-15 00:00:00+00:00  1185.000000  75.820000
2014-11-16 00:00:00+00:00  1185.000000  75.820000
2014-11-17 00:00:00+00:00  1183.000000  75.639999
2014-11-18 00:00:00+00:00  1196.699951  74.610001


/var/folders/pf/4hxrldrx2g7g5x19dwlvk36w0000gn/T/ipykernel_87298/1861915078.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gold_data_cleaned = gold_data_cleaned.fillna(method='ffill')
/var/folders/pf/4hxrldrx2g7g5x19dwlvk36w0000gn/T/ipykernel_87298/1861915078.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  oil_data_cleaned = oil_data_cleaned.fillna(method='ffill')
